![Information_Technology_University.png](attachment:Information_Technology_University.png)
<h1><center>Assignment 1</center></h1>
<h2><center>Building an Information Retrieval System</center></h2>
In this assignment we will explore a way to rank the documents according to given query to get relevant documents (kind of a search engine). 

In [1]:
# from google.colab import drive
# drive.mount('/content/drive')

In [2]:
# !cp '/content/drive/MyDrive/ITU/TTDS Assignment 1.zip' ./
# !unzip '/content/TTDS Assignment 1.zip' -d ./
# !mv /content/Assignment/data ./data/

In [3]:
# !wget -O stopwords.txt "https://gist.githubusercontent.com/sebleier/554280/raw/7e0e4a1ce04c2bb7bd41089c9821dbcf6d0c786c/NLTK's%2520list%2520of%2520english%2520stopwords"

##### <h4>Instructions:</h4>
- DO NOT PLAGIARISE (This may result in zero grade in this assignment and further penalty will be applied to the final grade)
- Follow the instructions given in the code cells that are written, not following the instruction can also result in penalization
- In case of any query, you can email at MSDS19087@itu.edu.pk
- Remember! We will enter a query and you should print the ranked list of document names, also display the contents of the top 5 documents
- Submit your Assignment as a .ipynb (python notebook) file with your roll number as the file name. Like **"MSDS19087.ipynb"**
- Do not submit data and output files. **only submit the notebook**
- Write user-defined functions for each task. You can not use any library functions (such as sklearn, torch, TensorFlow, nltk, spacy, or other natural language processing libraries for the tasks such as tf, df, idf, tf-idf, cosine similarity, etc) other than basic functions (such as numpy, pandas, os, math)
- Late penalty for this assignment and all other assignments is a deduction of 10% of obtained marks every 12 hours. In case of any emergency send an email before the deadline stating the issue with proof. The assignment will not be accepted after 3 days of the deadline.
- **Any part you see is taking time in processing:**
    - To start just take a small sample of the dataset (let's say 10% or even smaller)
    - When the code is working fine, run it for the whole dataset and **must** save the output in a csv/text file to avoid waste of time in re-run the code

### Task 1:
- import relevant python packages  

In [4]:
import os
import re
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
# write code to import other packages here

### Task 2
- Read the names of all text files given in the data folder
- Read all text files and save them in a list in a way that words of all files are saved in seperate lists
- All words must have all lower case letters
- Remove stop words (very common words such "this", "The", "is", you can search for stop words), punctuations marks or other special chracters or html tags (hint: use regular expression)

In [12]:
# Write code here
# Read all files and save their text into a dictionary
def read_all_files():
  parent = 'data/'
  file_names = os.listdir(parent)
  temp_data_dic = {}
  for fle in file_names:
      file_name = os.path.join(parent,fle)
      with open(file_name,'rb') as f:
          file_content = f.read()
          temp_data_dic[fle] = str(file_content)
        
  print("Total Files:",len(temp_data_dic))
  return temp_data_dic
org_data_dic = read_all_files()

Total Files: 3000


In [6]:
def load_stop_words():
    # Find all the Special characters
    SPECIAL_CHARS = ["!","\"","#","$","%","&","'","(",")","*","+",",","-",".","/",":",";","<","=",">","?","@","[","\\","]","^","_","`","{","|","}","~"]
    with open('stopwords.txt','rb') as f:
      # Read english stop words from a file
      STOP_WORDS = str(f.read())
      # Remove first two character and last 3 character as they are not part of the stop words
      STOP_WORDS = STOP_WORDS[2:-3]
      # Make a List of the stop words
      STOP_WORDS = STOP_WORDS.split('\\n')
    print('STOP_WORDS: ',STOP_WORDS)
    return (STOP_WORDS, SPECIAL_CHARS)

In [7]:
# Regex for special chars and stop words
def create_stopword_regex():
    stop_words, special_chars = load_stop_words()
    # create stop words regex
    # STOP_WORDS_REG = "(\\b"+"\\b)|(\\b".join(STOP_WORDS)+"\\b)"
    stop_word_reg="(\\b"+"\\b|\\b".join(stop_words)+"\\b)"
    print("Stop Word Regex:",stop_word_reg)

    # create special chars regex
    special_chars_reg = "("
    for char in special_chars:
      if len(char) < 2:
        special_chars_reg += f"\\{char}|"
      else:
        special_chars_reg += f"{char}|"
    special_chars_reg = special_chars_reg[0:-1]+")"
    print("Special Chars Regex:",special_chars_reg)

    return (stop_word_reg, special_chars_reg)

STOP_WORDS_REG, SPECIAL_CHARS_REG = create_stopword_regex()

STOP_WORDS:  ['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', 'her', 'hers', 'herself', 'it', 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so', 'than', 'too', 'very', 's', 't', 'can', 'will', 'just', 'don', 

In [8]:

def clean_content(content):
    global STOP_WORDS_REG, SPECIAL_CHARS_REG
    # Files contain some binary chars following regex is replacing those with a None
    content = re.sub(r'\\x([a-f]|[0-9]){2}','',content)
    # Remove all numbers
    content = re.sub(r'[0-9]*','',content)


    # Convert content of a file to lowercase
    content = content.lower()
    
    # remove \r, \n
    content = re.sub(r'\\r',' ',content)
    content = re.sub(r'\\n',' ',content)
    
    # Removing stop words using STOP_WORDS_REG and replacing with None
    content = re.sub(STOP_WORDS_REG,'',content)
    

    # Removing special chars
    content = re.sub(SPECIAL_CHARS_REG,' ',content)

        
    #Remove all character whom leng is less than 3
    content = re.sub(r'\b\w{1,2}\b',r' ',content)

    # Remove extra space
    content = re.sub(r'\s+',r' ',content)

    # Split string with space and return back
    return content.split()

In [14]:
# Iterate over all the content files, clean them and save clean word in word_data

word_data = {}

def clean_all_data():
  global word_data,org_data_dic

  temp = 0
  for fle, content in org_data_dic.items():
      word_data[fle] = clean_content(content)

      temp += 1
      if temp % 100 == 0:
        print(f"{temp} out of 3000 documents are cleaned")
clean_all_data()

100 out of 3000 documents are cleaned
200 out of 3000 documents are cleaned
300 out of 3000 documents are cleaned
400 out of 3000 documents are cleaned
500 out of 3000 documents are cleaned
600 out of 3000 documents are cleaned
700 out of 3000 documents are cleaned
800 out of 3000 documents are cleaned
900 out of 3000 documents are cleaned
1000 out of 3000 documents are cleaned
1100 out of 3000 documents are cleaned
1200 out of 3000 documents are cleaned
1300 out of 3000 documents are cleaned
1400 out of 3000 documents are cleaned
1500 out of 3000 documents are cleaned
1600 out of 3000 documents are cleaned
1700 out of 3000 documents are cleaned
1800 out of 3000 documents are cleaned
1900 out of 3000 documents are cleaned
2000 out of 3000 documents are cleaned
2100 out of 3000 documents are cleaned
2200 out of 3000 documents are cleaned
2300 out of 3000 documents are cleaned
2400 out of 3000 documents are cleaned
2500 out of 3000 documents are cleaned
2600 out of 3000 documents are cle

In [15]:
# Sample cleaned data
word_data[list(word_data.keys())[0]]

['probabilistic',
 'top',
 'parsing',
 'language',
 'modeling',
 'brian',
 'roark',
 'brown',
 'university',
 'paper',
 'describes',
 'functioning',
 'broad',
 'coverage',
 'probabilistic',
 'top',
 'parser',
 'application',
 'problem',
 'language',
 'modeling',
 'speech',
 'recognition',
 'paper',
 'first',
 'introduces',
 'key',
 'notions',
 'language',
 'modeling',
 'probabilistic',
 'parsing',
 'briefly',
 'reviews',
 'ome',
 'previous',
 'approaches',
 'tousing',
 'syntactic',
 'structure',
 'language',
 'modeling',
 'lexicalized',
 'probabilistic',
 'top',
 'parser',
 'presented',
 'performs',
 'well',
 'terms',
 'accuracy',
 'returned',
 'parses',
 'efficiency',
 'found',
 'relative',
 'best',
 'broad',
 'coverage',
 'statistical',
 'parsers',
 'new',
 'language',
 'model',
 'utilizes',
 'probabilistic',
 'top',
 'parsing',
 'outlined',
 'empirical',
 'results',
 'improves',
 'upon',
 'previous',
 'work',
 'test',
 'corpus',
 'perplexity',
 'interpolation',
 'trigram',
 'model',

In [16]:
# Create a list of lists containing words for each document
all_words_list = list(word_data.values())
print(len(all_words_list))

3000


### Task 3: Create a vocabulary 
- Get all unique words in all documents (corpus) and save them in a list 
- Get count of each word in whole corpus and save each word and its count in a dictionary
- Create a dictionary that assign each unique word a unique id (so that we can represent each word using a unique id)
- Create a dictionaty that assign each unique id a word (so that we can convert each id to back to the original word)
- Calculate some basic statistics too:
       - Total Number of Documents 
       - Total Number of Unique Words 
       - Total Number of Words in each Document 

In [17]:
def get_unique_words():
  # Create a single list of all words in all lists of documents
  all_words = []
  for t in all_words_list:
    all_words += t
  print(f"Total Words: {len(all_words)}")

  # Total Unique Words
  all_words = np.array(all_words)
  u_words = np.unique(all_words, return_counts= True)

  # Select only those word which occur more than 100 times
  uwrd_count = u_words[1][u_words[1] > 100]
  u_words = u_words[0][u_words[1] > 100]
  print(f"Total Unique Words:{len(u_words)}")
  return (u_words,uwrd_count)
  
unique_words, unique_words_counts = get_unique_words()
each_word_count = dict(zip(unique_words,unique_words_counts))

Total Words: 8945690
Total Unique Words:8380


In [18]:
# Get count of each word in whole corpus and save each word and its count in a dictionary
each_word_count = dict(zip(unique_words,unique_words_counts))

In [19]:
def assign_word_ids():
  # Assigning word ids
  word_to_id = {}
  id_to_word ={}

  ide = 0
  for wrd in unique_words:
    word_to_id[wrd] = ide
    id_to_word[ide] = wrd
    ide += 1
  return (word_to_id,id_to_word)

word_to_id,id_to_word = assign_word_ids()

In [21]:
# Total Number of Documents
# To calculate the total number of documents, we just need to find the length of all_words_list
# as it contain list of all documents words in their separate lists so
print(f"Total Number of Documents: {len(all_words_list)}")
print(f"Total Unique Words:{len(unique_words)}")

Total Number of Documents: 3000
Total Unique Words:8380


In [51]:
# Total Number of words in each document
doc_word_count = {}
for dn,dv in word_data.items():
    doc_word_count[dn] = len(dv)

document_count_df = pd.DataFrame({'Name':doc_word_count.keys(),'Count':doc_word_count.values()})
document_count_df.head(5)

,Name,Count
0,J01-2004.pdf.txt,7347
1,E87-1008.pdf.txt,2013
2,A94-1011.pdf.txt,2548
3,E09-1100.pdf.txt,3430
4,J04-2003.pdf.txt,6297


In [52]:
document_count_df[document_count_df.Name == "E87-1008.pdf.txt"]

,Name,Count
1,E87-1008.pdf.txt,2013


### Task 4
- Calculate term frequency (for each word in each document)
    - Calcuate how many times each word is repeated in each document 
    - Divide word count by total number of words in the document 
    - Assign zero if word doesnt appear in the doccument 
    - t is the term or selected word
![image.png](attachment:image.png)
    

In [22]:
def count_words(words,temp_counter):
    word_counter = temp_counter.copy()
    temp_arr = np.array(words)
    u_temp_arr = np.unique(temp_arr,return_counts=True)
    u_temp_dic = dict(zip(u_temp_arr[0],u_temp_arr[1]))
    for wrd in word_counter.keys():
      if wrd in u_temp_dic.keys():
        word_counter[wrd] = u_temp_dic[wrd]
    return word_counter

In [25]:

# Write code here

#First Create a dictionary with all unique words and their count equal to 0
temp_counter = {}
for wrd in unique_words:
    temp_counter[wrd] = 0

def create_count_dataframe(word_data,):
  global temp_counter
  # We will target only first 50 documents due to memory shortage
  # target_documents = list(word_data.keys())[:50]
  target_documents = list(word_data.keys())
  temp_tf_df = pd.DataFrame(index =unique_words,columns=target_documents)

  # Go to each targeted document and count it words
  for ind,t_doc in enumerate(target_documents):
      words =  word_data[t_doc]
      word_count = count_words(words,temp_counter)
      temp_tf_df[t_doc] = word_count.values()
      del word_count

      if ind % 100 == 0:
        print(f"{ind} out of 3000 documents cleaned.") 

  return temp_tf_df

tf_df = create_count_dataframe(word_data)

0 out of 3000 documents cleaned.
100 out of 3000 documents cleaned.
200 out of 3000 documents cleaned.
300 out of 3000 documents cleaned.
400 out of 3000 documents cleaned.
500 out of 3000 documents cleaned.
600 out of 3000 documents cleaned.
700 out of 3000 documents cleaned.
800 out of 3000 documents cleaned.
900 out of 3000 documents cleaned.
1000 out of 3000 documents cleaned.
1100 out of 3000 documents cleaned.
1200 out of 3000 documents cleaned.
1300 out of 3000 documents cleaned.
1400 out of 3000 documents cleaned.
1500 out of 3000 documents cleaned.
1600 out of 3000 documents cleaned.
1700 out of 3000 documents cleaned.
1800 out of 3000 documents cleaned.
1900 out of 3000 documents cleaned.
2000 out of 3000 documents cleaned.
2100 out of 3000 documents cleaned.
2200 out of 3000 documents cleaned.
2300 out of 3000 documents cleaned.
2400 out of 3000 documents cleaned.
2500 out of 3000 documents cleaned.
2600 out of 3000 documents cleaned.
2700 out of 3000 documents cleaned.
2800

In [26]:
# Count of every words in each document
tf_df.head(5)

,J01-2004.pdf.txt,E87-1008.pdf.txt,A94-1011.pdf.txt,E09-1100.pdf.txt,J04-2003.pdf.txt,D09-1086.pdf.txt,E03-1005.pdf.txt,D12-1044.pdf.txt,E06-1010.pdf.txt,E14-1064.pdf.txt,...,D15-1151.pdf.txt,D11-1048.pdf.txt,D12-1056.pdf.txt,E06-1045.pdf.txt,D09-1142.pdf.txt,E14-1036.pdf.txt,D15-1033.pdf.txt,A94-1022.pdf.txt,A92-1013.pdf.txt,E97-1049.pdf.txt
aaai,1,0,0,0,1,0,3,0,0,0,...,0,0,0,0,0,0,0,0,0,0
aachen,0,0,0,0,6,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
abbreviated,0,0,0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0
abbreviation,0,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
abbreviations,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [28]:
# Replace each word with its id
s = tf_df.index.to_series()
tf_df.index = s.map(word_to_id)
tf_df = tf_df.T
tf_df.head(5)

,0,1,2,3,4,5,6,7,8,9,...,8370,8371,8372,8373,8374,8375,8376,8377,8378,8379
J01-2004.pdf.txt,1,0,0,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
E87-1008.pdf.txt,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
A94-1011.pdf.txt,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
E09-1100.pdf.txt,0,0,0,1,0,0,0,0,0,2,...,0,0,11,1,0,0,0,0,0,0
J04-2003.pdf.txt,1,6,0,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0


In [29]:
# Calculate Term Frequncy by dividing the count of each term by total count
def calculate_term_frequency(t_df):
  tf_t_d = t_df.divide(t_df.sum(axis=1),axis=0)
  return tf_t_d
tf_t_d = calculate_term_frequency(tf_df)
tf_t_d.head(10)

,0,1,2,3,4,5,6,7,8,9,...,8370,8371,8372,8373,8374,8375,8376,8377,8378,8379
J01-2004.pdf.txt,0.000144,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.000144,...,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0
E87-1008.pdf.txt,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.000000,...,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0
A94-1011.pdf.txt,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.000000,...,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0
E09-1100.pdf.txt,0.000000,0.000000,0.0,0.000316,0.0,0.000000,0.0,0.0,0.0,0.000632,...,0.0,0.000000,0.003474,0.000316,0.0,0.000000,0.0,0.0,0.0,0.0
J04-2003.pdf.txt,0.000170,0.001021,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.000000,...,0.0,0.000000,0.000000,0.000000,0.0,0.000170,0.0,0.0,0.0,0.0
D09-1086.pdf.txt,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.000262,...,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0
E03-1005.pdf.txt,0.001211,0.000000,0.0,0.000000,0.0,0.000404,0.0,0.0,0.0,0.000000,...,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0
D12-1044.pdf.txt,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.000303,...,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0
E06-1010.pdf.txt,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.000490,...,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0
E14-1064.pdf.txt,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.000000,...,0.0,0.001903,0.000000,0.000000,0.0,0.001269,0.0,0.0,0.0,0.0


In [30]:
# document where words are most frequent
tf_t_d.max()
tf_t_d.idxmax()

0       J00-2012.pdf.txt
1       E14-2008.pdf.txt
2       A92-1037.pdf.txt
3       D14-1202.pdf.txt
4       J02-3002.pdf.txt
              ...       
8375    D11-1038.pdf.txt
8376    E06-2025.pdf.txt
8377    D09-1096.pdf.txt
8378    D09-1096.pdf.txt
8379    D13-1143.pdf.txt
Length: 8380, dtype: object

### Task 5
- Calculate document frequency (for each word)
    - Calculate the total number documents in which each word appears  
![image-2.png](attachment:image-2.png)

In [32]:
def cal_doc_frequency(t_df):
  # document count where each word occur
  doc_fr = (t_df > 0).sum(axis=0)
  # change id to word
  s = doc_fr.index.to_series()
  doc_fr.index = s.map(id_to_word)
  return doc_fr
doc_fr = cal_doc_frequency(tf_df)
doc_fr

aaai             611
aachen            40
abbreviated       84
abbreviation     114
abbreviations    141
                ... 
zhu              447
zollmann          39
zone              44
zones             25
zweig             64
Length: 8380, dtype: int64

### Task 6
- Calculate inverse document frequency for each word in the vocabulary
    - IDF is the inverse of the document frequency which measures the informativeness of term t
    - The formula to calculate the idf is as following 
    
![image.png](attachment:image.png)

In [33]:
# Write code here
def cal_idf(t_df):
  N = len(t_df.columns)
  df = t_df > 0
  df = df.sum(axis = 0)
  idf_t = np.log(N/(df+1))
  return idf_t
idf_t = cal_idf(tf_df)
idf_t

0       2.616871
1       5.320031
2       4.590952
3       4.288671
4       4.077776
          ...   
8375    2.928810
8376    5.344724
8377    5.226941
8378    5.775507
8379    4.859216
Length: 8380, dtype: float64

### Task 7
- Calculate tf-idf for each word of each document 
    - Formula to calculate the tf-idf is as following:
        ![image.png](attachment:image.png)

In [36]:
# Write code here
def cal_tf_idf(tf_t_d,idf_t):
  tf_idf_t_d = tf_t_d * idf_t
  return tf_idf_t_d
tf_idf_t_d = cal_tf_idf(tf_t_d,idf_t)
tf_idf_t_d.head(10)

,0,1,2,3,4,5,6,7,8,9,...,8370,8371,8372,8373,8374,8375,8376,8377,8378,8379
J01-2004.pdf.txt,0.000376,0.00000,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.000313,...,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0
E87-1008.pdf.txt,0.000000,0.00000,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.000000,...,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0
A94-1011.pdf.txt,0.000000,0.00000,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.000000,...,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0
E09-1100.pdf.txt,0.000000,0.00000,0.0,0.001355,0.0,0.000000,0.0,0.0,0.0,0.001377,...,0.0,0.000000,0.011592,0.001273,0.0,0.000000,0.0,0.0,0.0,0.0
J04-2003.pdf.txt,0.000445,0.00543,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.000000,...,0.0,0.000000,0.000000,0.000000,0.0,0.000498,0.0,0.0,0.0,0.0
D09-1086.pdf.txt,0.000000,0.00000,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.000572,...,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0
E03-1005.pdf.txt,0.003168,0.00000,0.0,0.000000,0.0,0.002075,0.0,0.0,0.0,0.000000,...,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0
D12-1044.pdf.txt,0.000000,0.00000,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.000660,...,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0
E06-1010.pdf.txt,0.000000,0.00000,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.001068,...,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0
E14-1064.pdf.txt,0.000000,0.00000,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.000000,...,0.0,0.004937,0.000000,0.000000,0.0,0.003716,0.0,0.0,0.0,0.0


### Task 8
- Create a sparse matrix that stores the frequency of each word in each document (this is something similar to what you have already done)
![Document_Term.JPG](attachment:Document_Term.JPG)

In [37]:
# Sparse matrix code is already written above
tf_df.head(10)

,0,1,2,3,4,5,6,7,8,9,...,8370,8371,8372,8373,8374,8375,8376,8377,8378,8379
J01-2004.pdf.txt,1,0,0,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
E87-1008.pdf.txt,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
A94-1011.pdf.txt,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
E09-1100.pdf.txt,0,0,0,1,0,0,0,0,0,2,...,0,0,11,1,0,0,0,0,0,0
J04-2003.pdf.txt,1,6,0,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
D09-1086.pdf.txt,0,0,0,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
E03-1005.pdf.txt,3,0,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
D12-1044.pdf.txt,0,0,0,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
E06-1010.pdf.txt,0,0,0,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
E14-1064.pdf.txt,0,0,0,0,0,0,0,0,0,0,...,0,6,0,0,0,4,0,0,0,0


### Task 9: Query matching using Cosine Similarity
- Take a search query from user as an input 
- Treat query as a document and:
    - Apply same preprocessing to the query
    - Calculate term frequencies in query
- Now use cosine similarity to calculate the similaraity of a document to the given query (use query (term frequencies) vector and sparse matrix containing term frequencies)
- Sort cosine similarities with all given documents to get most relevant documents 
- Display top 5 relevant documents and explore if they are actually relevant or not

In [ ]:
text = input("Enter the Query text:")

In [39]:
# Write code here
with open('data/A00-1000.pdf.txt','rb') as f:
  text = str(f.read())

### Cosine Similarity on Sparse Matrix

In [40]:
# Write code here

# Clearn the data from test document and convert it into a dataframe
def create_query_df(text):
  words = clean_content(text)
      
  tdf = pd.DataFrame({'test':count_words(words,temp_counter)})
  s = tdf.index.to_series()
  tdf.index = s.map(word_to_id)
  tdf=tdf.T
  return tdf
tdf = create_query_df(text)
tdf

,0,1,2,3,4,5,6,7,8,9,...,8370,8371,8372,8373,8374,8375,8376,8377,8378,8379
test,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [41]:
def find_cosine_similarity_for_sparse(tf_df,tdf):
  indexes = tf_df.index.to_numpy()
  cs_data = {}
  # t = 0
  for index in indexes:
      d1 = tdf.loc['test'].to_numpy()
      d2 = tf_df.loc[index].to_numpy()
      # print(np.sum(d1),np.sum(d2),index)
      cosine_similarity = np.sum(d1*d2)/(np.sqrt(np.sum(np.square(d1)))*np.sqrt(np.sum(np.square(d2))))
      cs_data[index] = cosine_similarity
      
  cs_df = pd.DataFrame({'cosine_similarity_with_test':cs_data})
  cs_df = cs_df.sort_values(by=['cosine_similarity_with_test'], ascending=False)
  return cs_df
cs_df = find_cosine_similarity_for_sparse(tf_df,tdf)
cs_df.head(5)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:9: RuntimeWarning: invalid value encountered in true_divide
  if __name__ == '__main__':


,cosine_similarity_with_test
A00-1000.pdf.txt,1.000000
A97-1000.pdf.txt,0.627260
A92-1000.pdf.txt,0.492024
A88-1000.pdf.txt,0.482359
A94-1000.pdf.txt,0.480977


### Cosine Similarity on Frequency Vector

In [42]:
def find_cosine_similarity_for_freq_vector(tf_t_d,tdf):
  tdf_t_d = tdf.divide(tdf.sum(axis=1),axis=0)
  indexes = tf_t_d.index.to_numpy()
  cs_data = {}
  # t = 0
  for index in indexes:
      d1 = tdf_t_d.loc['test'].to_numpy()
      d2 = tf_t_d.loc[index].to_numpy()
      # print(np.sum(d1),np.sum(d2),index)
      cosine_similarity = np.sum(d1*d2)/(np.sqrt(np.sum(np.square(d1)))*np.sqrt(np.sum(np.square(d2))))
      cs_data[index] = cosine_similarity
      
  cs_df = pd.DataFrame({'cosine_similarity_with_test':cs_data})
  cs_df = cs_df.sort_values(by=['cosine_similarity_with_test'], ascending=False)
  return cs_df
cs_df = find_cosine_similarity_for_freq_vector(tf_t_d,tdf)
cs_df.head(5)

,cosine_similarity_with_test
A00-1000.pdf.txt,1.000000
A97-1000.pdf.txt,0.627260
A92-1000.pdf.txt,0.492024
A88-1000.pdf.txt,0.482359
A94-1000.pdf.txt,0.480977


### Task 10: Query matching using Vector Space Model
- Calculate the similarities using vector space model in following way:
    - Apply same preprocessing to the query
    - Calculate the tfidf of the query in the same way (here term frequencies are the calculated using query while idf scores are same as calculated above)
    - Use cosine similarity between documents and query (based on tfidf) and rank the documents according to the similarity score 
- Display top 5 relevant documents and explore if they are actually relevant or not

In [44]:
# Write code here
def find_idf_query(tdf,idf_t):
  tdf_t_d = tdf.divide(tdf.sum(axis=1),axis=0)
  tdfidf_t_d = tdf_t_d * idf_t
  return tdfidf_t_d

tdfidf_t_d = find_idf_query(tdf,idf_t)
tdfidf_t_d

,0,1,2,3,4,5,6,7,8,9,...,8370,8371,8372,8373,8374,8375,8376,8377,8378,8379
test,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [49]:

def find_cosine_similarity_for_idf(tf_idf_t_d,tdfidf_t_d):
  indexes = tf_idf_t_d.index.to_numpy()
  cs_data = {}
  # t = 0
  for index in indexes:
      d1 = tdfidf_t_d.loc['test'].to_numpy()
      d2 = tf_idf_t_d.loc[index].to_numpy()
      # print(np.sum(d1),np.sum(d2),index)
      cosine_similarity = np.sum(d1*d2)/(np.sqrt(np.sum(np.square(d1)))*np.sqrt(np.sum(np.square(d2))))
      cs_data[index] = cosine_similarity
      
  cs_df = pd.DataFrame({'cosine_similarity_with_test':cs_data})
  cs_df = cs_df.sort_values(by=['cosine_similarity_with_test'], ascending=False)
  return cs_df

cs_df = find_cosine_similarity_for_idf(tf_idf_t_d,tdfidf_t_d)
cs_df.head(5)


,cosine_similarity_with_test
A00-1000.pdf.txt,1.000000
A97-1000.pdf.txt,0.423739
A00-4000.pdf.txt,0.374939
A94-1000.pdf.txt,0.299330
E99-1000.pdf.txt,0.277045
